In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/minute-weather/minute_weather.csv


<p style="font-family: Arial; font-size:3.75vw;color:purple; font-style:bold"><br>
Clustering Exercise Notebook
</p><br>

# Exercise Notebook Instructions

### 1. Important: Only modify the cells which instruct you to modify them - leave "do not modify" cells alone.  

The code which tests your responses assumes you have run the startup/read-only code exactly.

### 2. Work through the notebook in order.

Some of the steps depend on previous, so you'll want to move through the notebook in order.

### 3. It is okay to use numpy libraries.

You may find some of these questions are fairly straightforward to answer using built-in numpy functions.  That's totally okay - part of the point of these exercises is to familiarize you with the commonly used numpy functions.

### 4. Seek help if stuck

If you get stuck, don't worry!  You can either review the videos/notebooks from this week, ask in the course forums, or look to the solutions for the correct answer.  BUT, be careful about looking to the solutions too quickly.  Struggling to get the right answer is an important part of the learning process.

In [2]:
# DO NOT MODIFY

# import appropriate libraries

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [3]:
# DO NOT MODIFY

# We will use the minute weather dataset for this exercise.

def get_data():
    return pd.read_csv('../input/minute-weather/minute_weather.csv')

df = get_data()

<p style="font-family: Arial; font-size:2.75vw;color:purple; font-style:bold"><br>

Exercise 1: Sampling Down a Time Series Dataset<br><br></p>


In the cell below, modify the function to RETURN a new dataframe that is 
sampled down by taking every k row only. For example, if k = 5, the 
function will take every 5th row and skip rows in between.

The inputs to function are a dataframe and an integer k.

In [7]:
# modify this cell

def down_sample(df, k):
    ### BEGIN SOLUTION
    df_sampled = df[df['rowID'] % k == 0]
    return df_sampled
    ### END SOLUTION

In [8]:
# DO NOT MODIFY
ans = 15873

try:
    sampled_df = down_sample(df, 100)
    assert np.alltrue(sampled_df.shape[0] == ans)
except AssertionError as e: print("Try again, your output did not match the expected answer above")

<p style="font-family: Arial; font-size:2.75vw;color:purple; font-style:bold"><br>

Exercise 2: Data Cleaning<br><br></p>


In the cell below, modify the function to do following tasks and RETURN a new data frame: (do not modify the input dataframe) <br><br>

- delete column 'rain_accumulation'
- delete column 'rain_duration'
- delete rows that contain atleast one NULL

In [14]:
# modify this cell

def clean_data(df):
    ### BEGIN SOLUTION
    df_copy = df.copy()
    del df_copy['rain_accumulation']
    del df_copy['rain_duration']
    df_copy = df_copy.dropna()
    return df_copy
    ### END SOLUTION

In [15]:
# DO NOT MODIFY

try: 
    cleaned_df = clean_data(sampled_df)
    assert np.alltrue(sampled_df.shape == (15873, 13))
    assert np.alltrue(cleaned_df.shape == (15870, 11))
except AssertionError as e: print("Try again ! Check both  clean_data and down_sample functions")

In [16]:
cleaned_df.describe()

,rowID,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,min_wind_direction,min_wind_speed,relative_humidity
count,1.587000e+04,15870.000000,15870.000000,15870.000000,15870.000000,15870.000000,15870.000000,15870.000000,15870.000000,15870.000000
mean,7.936240e+05,916.829464,61.857384,161.287524,2.792804,162.700945,3.414625,166.644297,2.152268,47.591084
std,4.582191e+05,3.051662,11.834858,95.313161,2.070506,92.269601,2.428906,97.824836,1.758114,26.189680
min,0.000000e+00,905.100000,32.360000,0.000000,0.100000,0.000000,0.100000,0.000000,0.000000,1.500000
25%,3.968250e+05,914.800000,52.700000,61.000000,1.300000,68.000000,1.600000,75.000000,0.800000,24.700000
50%,7.936500e+05,916.700000,62.420000,182.000000,2.200000,186.000000,2.800000,180.000000,1.700000,44.600000
75%,1.190375e+06,918.700000,70.880000,217.000000,3.800000,223.000000,4.600000,212.000000,3.000000,67.900000
max,1.587200e+06,929.400000,96.440000,359.000000,20.100000,359.000000,20.900000,359.000000,19.500000,92.900000


<p style="font-family: Arial; font-size:2.75vw;color:purple; font-style:bold"><br>

Exercise 3: Perform Scaling of input data<br><br></p>

In the cell below, modify the function that takes three inputs: a dataframe D, a list of features F
<br>
The function should do the following:
<br>
- Pick out columns corresponding to features F from dataframe D
- Use StandardScaler to perform scaling and return the scaled data frame

In [22]:
# modify this cell

def scale_task(D, F):
    ### BEGIN SOLUTION
    select_df = D[F]
    X = StandardScaler().fit_transform(select_df)
    return pd.DataFrame(X, columns=select_df.columns)
    ### END SOLUTION

In [23]:
# DO NOT MODIFY

try: 
    features = ['air_pressure', 'air_temp', 'relative_humidity']
    scaled_data = scale_task(cleaned_df, features)
    assert np.allclose(scaled_data.mean(axis=0), 0)
    assert np.allclose(scaled_data.std(axis=0),1, atol=1e-04)
except AssertionError as e: print("Try again, your solution did not produce the expected output above")

<p style="font-family: Arial; font-size:2.75vw;color:purple; font-style:bold"><br>

Exercise 4: Perform Clustering<br><br></p>

In the cell below, modify the function that takes two inputs: a dataframe D (already scaled using your previous function), and an integer for # of clusters to create N.
<br>
The function should do the following:
<br>
- Perform clustering using your algorithm of choice
- Create N clusters
- Return a **dataframe** of N rows, where each row is a cluster center.

In [27]:
# modify this cell

def clustering_task(D, N):
    ### BEGIN SOLUTION
    kmeans = KMeans(n_clusters=N)
    model = kmeans.fit(D)
    centers = model.cluster_centers_
    return pd.DataFrame(centers, columns=D.columns)
    ### END SOLUTION

In [28]:
# DO NOT MODIFY

try: 
    ct = clustering_task(scaled_data, 3)
    assert np.alltrue(ct.shape == (3, 3))
except AssertionError as e: print("Keep trying - ensure all previous functions also work correctly ")

try:
    print(40*'-')
    print(ct.describe())
except AssertionError as e: print("Ensure clustering_task returns a Pandas dataframe")

----------------------------------------


AttributeError: 'numpy.ndarray' object has no attribute 'describe'